# Lecture 5: Operations on Inductively Defined Structures

Previously, we examined inductive definitions starting with numbers, lists, binary trees, arithmetic expressions and a simple imperative language. However, beyond defining them, we did not do much else with them. In this lecture, we will fix this by performing a variety of exciting operations on inductively defined structures we have defined so far.

We will examine two mechanisms for defining these operations: 
1. Using a *visitor pattern*: i.e, implement the operation as a member function as we will see.
2. Using pattern matching: this is a special feature of functional languages like Scala, Lisp, Caml and Haskell. Although you will find libraries in Python (https://github.com/grantjenks/python-pattern-matching) and C++ (https://github.com/solodon4/Mach7) that can mimic this very powerful feature. 

The first option is generically applicable to most languages. The second one is very special and very powerful. We will focus extensively on the second option of pattern matching, while mentioning what a visitor pattern is.

## Operations on Numbers

Let us recall the grammar for inductively defining numbers.

$$\textbf{NatNum} \ \rightarrow\ Z\; |\; Succ(\textbf{NatNum}) $$


In [1]:
sealed trait NatNum

case object Z extends NatNum

case class Succ(n : NatNum) extends NatNum


defined trait NatNum
defined object Z
defined class Succ

The simplest function we can imagine is to add one to a given NatNum. This is easy to write since this is exactly what *Succ* does.

In [2]:
def addOne(n:NatNum) = Succ(n)

defined function addOne

In [3]:
val two = Succ(Succ(Z))
val three = addOne(two)

two: Succ = Succ(Succ(Z))
three: Succ = Succ(Succ(Succ(Z)))

Now we wish to write a function _minusOne_ that given a number subtracts one from it. Before we do so, we have to understand how to handle the zero case. We could raise an error/exception saying that it is undefined. This is the best way to do it since it is the most honest.

Here is how we should do it. 

~~~
minusOne(s) =  if s is of the form Succ(t) then return t, else s must be Z and return Error
~~~

Therefore we need a construct that checks if a given input NatNum is of the form Succ(t) and extracts this inner stuff t, but how?

There are two solutions to this.  First is to redefine things to have the _minusOne_ function implemented inside each class. This solves the problem using the way object oriented programs work.





In [4]:
sealed trait NatNum1 {
    def minusOne(): NatNum1 // All those who inherit from NatNum1 better implement this
                            // minusOne function, or else ... 
}

case object Z1 extends NatNum1{
    def minusOne(): NatNum1  = { // subtracting one from Zero should give an error.
        throw (new IllegalArgumentException("minusOne cannot be called on Zero"))
    }
}

case class Succ1(n: NatNum1) extends NatNum1 {
    def minusOne(): NatNum1 =  { // Otherwise, number is of the form Succ1(..) 
        return this.n // return the "inner stuff"
    }
}

defined trait NatNum1
defined object Z1
defined class Succ1

NatNum1 is the base class (it is an abstract class or a trait in scala). What this means is that any class that inherits from it must have all the members that are defined in it. We define a member minusOne() corresponding to the function we wish to implement. Therefore, when we call the minusOne function on an instance of NatNum1, the instance can be either a Z1 or a Succ1 class. In either case, the object system in scala ensures that the right function gets called. This is an indirect but effective way of finding out the question if a given NatNum1 is of the form Z1 or Succ1(t).

In [5]:
val t1 = Z1
val t2 = Succ1(Succ1(Succ1(Z1)))
val t3 = t2.minusOne()


t1: Z1 = Z1
t2: Succ1 = Succ1(Succ1(Succ1(Z1)))
t3: NatNum1 = Succ1(Succ1(Z1))

In [6]:
val t4 = t1.minusOne() // This will throw an exception

: 

The second way to do it is using the idea of *pattern matching*: a very powerful feature that is available in some functional programming languages including scala. Here is how it works.

An instance object of type NatNum can be of two forms Succ(t) or Z. Scala provides a construct very similar to the case statement in C like languages, but much more powerful.

In [7]:
def minusOne(num: NatNum): NatNum = {
    num match {
        case Succ(t) => t // Is n of the form Succ(t), then return t
        // The magic here is that the variable t gets assigned to the contents of num.n in this case.
        case Z => throw new IllegalArgumentException("minusOne cannot be called on Zero")
        // Is n of the form Z, then throw an exception
    }
}

defined function minusOne

In [8]:
val t5 = minusOne(Succ(Succ(Succ(Z))))

t5: NatNum = Succ(Succ(Z))

In [9]:
val t6 = minusOne(Z) // Boom!

: 

Let us write code to add two NatNum. The basic idea is this: 
- If the first argument to the call is of the form Z, then the answer is the second argument since 0 + something = something.
- If the first argument is of the form Succ(t) then simply make a recursive call to add t with Succ(second argument). We are simply saying (1 + t) + n = t + (1 + n)

Let us see how pattern matching can help us do this.

In [10]:
def addNatNums (n1: NatNum, n2: NatNum ): NatNum = {
    n1 match {
        case Z => n2  // If n1 == Z, then the result is just n2
        case Succ(t) => addNatNums(t, Succ(n2)) // Otherwise, the result is to peel of a Succ from first argument 
        // and add it to the second argument.
    }
}

defined function addNatNums

In [11]:
val two = Succ(Succ(Z))
val three = addOne(two)
val five = addNatNums(two, three)
val ten = addNatNums(five, five)

two: Succ = Succ(Succ(Z))
three: Succ = Succ(Succ(Succ(Z)))
five: NatNum = Succ(Succ(Succ(Succ(Succ(Z)))))
ten: NatNum = Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Z))))))))))

For curiosity, how would we implement it using the _visitor pattern_ ? Let us now redefine NatNum1 to require a new member function addNatNums. You can see how instead of pattern matching, we simply write the code for the zero case inside the object Z1 and the code for the successor case inside the object Succ1. The idea is the same as before but the two cases get split into two different member functions of Z1 and Succ1, respectively.

In [12]:
sealed trait NatNum1 {
    def minusOne(): NatNum1
    def addNatNums(n1: NatNum1): NatNum1
}

case object Z1 extends NatNum1{
    def minusOne(): NatNum1  = {
        throw (new IllegalArgumentException("minusOne cannot be called on Zero"))
    }
    
    def addNatNums(n1: NatNum1): NatNum1 = n1 // return is optional
    
}

case class Succ1(n: NatNum1) extends NatNum1 {
    def minusOne(): NatNum1 =  {
        return this.n
    }
    def addNatNums(n1: NatNum1): NatNum1 =  this.n.addNatNums(Succ1(n1)) // return is optional
    
}

defined trait NatNum1
defined object Z1
defined class Succ1

In [13]:
val two = Succ1(Succ1(Z1))
val three = Succ1(two)
val five = two.addNatNums(three)
val ten = five.addNatNums(five)

two: Succ1 = Succ1(Succ1(Z1))
three: Succ1 = Succ1(Succ1(Succ1(Z1)))
five: NatNum1 = Succ1(Succ1(Succ1(Succ1(Succ1(Z1)))))
ten: NatNum1 = Succ1(
  Succ1(Succ1(Succ1(Succ1(Succ1(Succ1(Succ1(Succ1(Succ1(Z1)))))))))
)

Once we have addition, multiplication can now be implemented using recursion, as below.

In [14]:
def multiplyNatNums(n1: NatNum, n2: NatNum ): NatNum = {
    n1 match {
        case Z => { return Z }
        case Succ(t) => { 
            // (t+1)* n2 = t * n2 + n2
            val s1 = multiplyNatNums(t, n2) // t * n2
            addNatNums(n2, s1) // n2 + t * n2 = (t+1)* n2 = n1 * n2!
        }
    }
}

defined function multiplyNatNums

In [15]:
val four = Succ(Succ(Succ(Succ(Z))))
val five = addOne(four)
val twenty = multiplyNatNums(five, four)
val hundred = multiplyNatNums(five, twenty)

four: Succ = Succ(Succ(Succ(Succ(Z))))
five: Succ = Succ(Succ(Succ(Succ(Succ(Z)))))
twenty: NatNum = Succ(
  Succ(
    Succ(
      Succ(
        Succ(
          Succ(
            Succ(
              Succ(
                Succ(
                  Succ(
                    Succ(
                      Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Z)))))))))
                    )
                  )
                )
              )
            )
          )
        )
      )
    )
  )
)
hundred: NatNum = Succ(
  Succ(
    Succ(
      Succ(
        Succ(
          Succ(
            Succ(
              Succ(
                Succ(
                  Succ(
                    Succ(
                      Succ(
                        Succ(
                          Succ(
                            Succ(
                              Succ(
                                Succ(
                                  Succ(
                                    Succ(
                                      S

## Operations on List of Numbers

Recall that we previously defined a grammar for lists.
$$\begin{array}{ccccc}
\textbf{NumList} & \rightarrow & Nil &\ |\  & Cons(\textbf{Num}, \textbf{NumList}) \\
\textbf{Num} & \rightarrow & 0 \ |\ 1\ |\ 2\ |\ 3\ |\ 4\ |\ \cdots \\
\end{array}$$

In [16]:
sealed trait NumList

case object Nil extends NumList

case class Cons(hd: Int, tl: NumList) extends NumList

defined trait NumList
defined object Nil
defined class Cons

There are many exciting things we wish to do to lists. The simplest one is to find the length of a list.
How do we do that in principle?
- The length of the empty list Nil is zero
- The length of the list of the form Cons(something, tail) is 1 + length(tail)

Let us use pattern matching to implement this.

In [17]:
def listLength(lst: NumList): Int = { lst match { // pattern match the lst
    case Nil => 0
    case Cons(_, tl) => 1 + listLength(tl) // _ here is to tell scala that I do not care what is the element.
    }
}

defined function listLength

How would we do it using a visitor? Simple, the listLength function is going to become a member of the trait 
NumList and get implemented in all the classes that inherit from it.

In [18]:
sealed trait AltNumList{
    def listLength(): Int
}

case object AltNil extends AltNumList {
    def listLength(): Int = 0
}

case class AltCons(hd: Int, tl: AltNumList)  extends AltNumList {
    def listLength(): Int = {
        1  + tl.listLength()
    }
}

defined trait AltNumList
defined object AltNil
defined class AltCons

In [19]:
val l1 = AltCons(1, AltCons(3, AltCons(7, AltNil)))
val j1 = l1.listLength()

l1: AltCons = AltCons(1, AltCons(3, AltCons(7, AltNil)))
j1: Int = 3

You may now be wondering why we are bothering describing both visitor functions and pattern matches. Aren't they just two different ways of achieving the same effect? The answer to that is yes for about 90% of the cases, but not always. Pattern matching can make life infinitely more easier. 


I would like to write a function now that does the following: given a list, check if it is sorted in ascending order.


In [20]:
def isAscendingOrder (l: NumList): Boolean = l match {
    
    case Nil => true // An empty list is ascending sure!
    
    case Cons(_, Nil) => true // A list with just one element is surely ascending ordered
    
    case Cons(j1, tl @ Cons(j2, _)) => (j1 <= j2) && isAscendingOrder(tl) 
    // We did something funky:
    // We pattern matched the first two elements of the list to j1 and j2 respectively.
    // Also, we told scala to call Cons(j2, _) by the name tl using the @ symbol. 
    // This is called pattern matching with names.
    
    
    case _ => { assert(false); false } // This is the catch all case and should never happen.
    
}

defined function isAscendingOrder

In [21]:
val b1 = isAscendingOrder( Cons(1, Cons(3, Cons(3, Cons(5, Cons(10, Nil ))))))

b1: Boolean = true

In [22]:
val b2 = isAscendingOrder( Cons(5, Cons(3, Cons(3, Cons(5, Cons(10, Nil ))))))

b2: Boolean = false

In [23]:
val b3 = isAscendingOrder( Cons(0, Cons(3, Cons(3, Cons(5, Cons(4, Nil ))))))

b3: Boolean = false

Things can be made even more funky with pattern matching. But this is where you should read a tutorial on pattern matching to acquaint yourself with all the funky features: https://docs.scala-lang.org/tour/pattern-matching.html

In [24]:
def isAscendingOrderAlt (l : NumList) : Boolean = l match {
    case Nil => true
    
    case Cons(_, Nil) => true
    
    case Cons(j1, Cons(j2, _)) if (j1 > j2) => false // This case matches only if j1 > j2
    
    case Cons(_, tl) => isAscendingOrderAlt(tl) // We can reach this case only if j1 <= j2, 
                                                // so there is no need to expand tl out
    
    // A catch all case is not needed: can you argue why?
}

defined function isAscendingOrderAlt

In [25]:
val b4 = isAscendingOrderAlt(Cons(1, Cons(3, Cons(3, Cons(5, Cons(10, Nil ))))))

b4: Boolean = true

In [26]:
val b5 =  isAscendingOrderAlt( Cons(0, Cons(3, Cons(3, Cons(5, Cons(4, Nil ))))))

b5: Boolean = false

Pattern matching really helps us in this example: *isAscendingOrder* is not easy to implement as a visitor. It will require something messy such as storing the previous element of the array in a global variable, extra function argument or doing something akin to pattern matching to try and extract the next array element. Can you try some of these options?

One nice operation on list is to reverse the list. 

In [27]:
// First consider a helper function that will take the list l and append an element e to its end
def appendToEndHelper(l: NumList, e: Int): NumList = l match { 
    case Nil => Cons(e, Nil)
    case Cons(j, tl) => Cons(j, appendToEndHelper(tl, e))
}

// Use the helper function appendToEndHelper to now define reverse
def reverseBad(l: NumList): NumList = l match {
    case Nil => Nil // Reverse of an empty list is empty
    case Cons(n1, t1) => {
        val r1 = reverseBad(t1) // reverse the tail
        appendToEndHelper(r1, n1) // append n1 to the end.
    }
}

defined function appendToEndHelper
defined function reverseBad

In [28]:
val l1 = Cons(1, Cons(3, Cons(5, Cons(7, Nil))))
val l2 = reverseBad(l1)

l1: Cons = Cons(1, Cons(3, Cons(5, Cons(7, Nil))))
l2: NumList = Cons(7, Cons(5, Cons(3, Cons(1, Nil))))

Here is the key question you should be asking. Why is the reverse method we wrote previously "Bad"?  The answer has to do with its complexity. It is often hard to understand what the complexity is when you write recursive functions. But imagine the same logic carried out on a linked list with a for loop. Can you now guess what the complexity would be on a list of length $n$?

If you guessed $\Theta(n^2)$, you are absolutely right. But this is a pity since we would like to reverse a list in time $\Theta(n)$. How can we achieve that?

Let us write a helper function with two arguments: the first argument is the tail of the list that we are yet to reverse, and the other argument is the reverse of the prefix of the list. Initially, the tail to be reversed is the entire list and the prefix is just Nil or the empty list. But as we "peel off" each element of the list, it joins the front of the reverse list so far. 

In [29]:
def reverseHelper(l: NumList, resultSoFar: NumList): NumList = l match {
    case Nil => resultSoFar
    case Cons(n, tail) => {
        val v1 = Cons(n, resultSoFar)
        return reverseHelper(tail, v1 )
    }
}
// This will now run in \Theta(n^2) rather than \Theta(n) time
def reverseGood(l: NumList): NumList = reverseHelper(l, Nil)

defined function reverseHelper
defined function reverseGood

In [30]:
val l3 = reverseGood(l1)

l3: NumList = Cons(7, Cons(5, Cons(3, Cons(1, Nil))))

## Operations on Trees with Numbers

Let us now get to the grammar of trees with numbers. Recall the grammar

$$\begin{array}{rclclcl}
\textbf{NumTree} & \rightarrow & Leaf & \ |\ & Node(\textbf{Num}, \textbf{NumTree}, \textbf{NumTree}) \\
\textbf{Num} & \rightarrow & 0 \ |\ 1\ |\ 2\ | \ 3 \ |\ \cdots \\
\end{array}$$


In [31]:
sealed trait NumTree
case object Leaf extends NumTree
case class Node(n: Int, left: NumTree, right: NumTree) extends NumTree

defined trait NumTree
defined object Leaf
defined class Node

Just like we defined length for a list, you can define functions like *depth* of a tree, *number* of leaves in a tree, *number* of nodes and so on.

But let us do something more interesting here.  We would like to check if a tree has the binary search tree property. 

Recall this property that says that for every node all the elements of its left subtree (if not a leaf) must be smaller than the node and all right subtree elements must be larger than the node.

In [32]:
// A simple attempt is to check at each node if its left child is smaller and right child is larger.
// This is wrong: you should be able to come up with an example why
def isBST_WrongCode(t: NumTree): Boolean = t match {
    case Leaf => true // Nothing to say for a leaf
    case Node(n, Leaf, Leaf) => true // Nothing to check since both children are leaves
    case Node(n1, Node(n2, _, _ ), _ ) if (n2 > n1) => false // Violates BST property 
    case Node(n1, _, Node(n3, _, _) )  if (n3 < n1) => false // Violates BST property
    case Node(n1, leftChild, rightChild) => isBST_WrongCode(leftChild) && isBST_WrongCode(rightChild) // The previous two cases did not match
    // Therefore, we conclude that n2 <= n1 and n3 >= n1, so all that remains is to 
    // check BST property of left and right children.
}

defined function isBST_WrongCode

In [33]:
def findMaxElementHelper(t: NumTree): Int = t match {
    case Leaf => Int.MinValue // Make it a smallest number 
    case Node(n, t1, t2) => { val v1 = findMaxElementHelper(t1) 
                              val v2 = findMaxElementHelper(t2) 
                              return List(n, v1, v2).max } 
}

def findMinElementHelper(t: NumTree): Int = t match {
    case Leaf => Int.MaxValue 
    case Node(n, t1, t2) => {
        val v1 = findMinElementHelper(t1)
        val v2 = findMinElementHelper(t2)
        return List(n, v1, v2).min
    }
    
}

def isBST(t: NumTree): Boolean = t match {
    case Leaf => true
    case Node(n1, lChild, rChild) => {
        val maxLeft = findMaxElementHelper(lChild)
        val minRight = findMinElementHelper(rChild)
        isBST(lChild) && isBST(rChild) && 
        (n1 >= maxLeft) && (n1 <= minRight)
    }
}

defined function findMaxElementHelper
defined function findMinElementHelper
defined function isBST

In [34]:
val tree1 = Node(1, Node(2, Node(3, Leaf, Leaf), Node(2, Leaf, Leaf)), Node(4, Leaf, Leaf))
val tree2 = Node(6, Node(2, Node(1, Leaf, Leaf), Leaf), Node(10, Node(7, Leaf, Leaf), Leaf))

tree1: Node = Node(
  1,
  Node(2, Node(3, Leaf, Leaf), Node(2, Leaf, Leaf)),
  Node(4, Leaf, Leaf)
)
tree2: Node = Node(
  6,
  Node(2, Node(1, Leaf, Leaf), Leaf),
  Node(10, Node(7, Leaf, Leaf), Leaf)
)

In [35]:
val b1 = isBST(tree1)

b1: Boolean = false

In [36]:
val b2 = isBST(tree2)

b2: Boolean = true

We still have a problem here. This time it is a problem of efficiency. The `isBST` method we wrote is quite
inefficient since it performs two passes over the tree. In the "outer pass" it visits each node in the tree and
then at each node, it performs yet another pass to find max and min elements of left and right subtrees.

What is the complexity of this approach: look at it from the point of view of each node. It is visited once in the outer iteration and for the inner loop, it is visited once for each parent. Thus the number of visits for each node is 1 + height of the node. Thus, total work done is the sum of the heights of all nodes in the tree. In the worst case this is $\Theta(n^2)$ worst case being achieved for the completely unbalanced binary tree.

Thus, we face the familiar problem we did when we tried to reverse a list: correct but inefficient code.

Let us make it efficient: how? 

The idea is to keep a bit of extra information for each subtree as we descend down in the outer iteration. Suppose we are at the root, we have no limits on what the values of the individual nodes can be. Thus, we start at the root 
and allow nodes to be in the interval $(-\infty, \infty)$. Suppose the root had the number $r$, we know that 
everything to its left can only be in the interval $(-\infty, r]$ and the right subtree in $[r, \infty)$.
If we keep going, at any node, we get an interval $[l, u]$ of allowable values. If the node value $v$ lies in that
range, we know that the left subtree must be in the range $[l, v]$ and right subtree in $[v,r]$. This gives us the plan to move forward with a helper function.

In [37]:
def isBST_Eff_Helper(t: NumTree, l: Int, r: Int): Boolean = t match {
    case Leaf => true
    case Node(j, lChild, rChild) if (l <= j && j <= r) => { 
        isBST_Eff_Helper(lChild, l, j) && isBST_Eff_Helper(rChild, j, r)
    } 
    case _ => false
}

def isBST_Efficient(t: NumTree): Boolean = isBST_Eff_Helper(t, Int.MinValue, Int.MaxValue)

defined function isBST_Eff_Helper
defined function isBST_Efficient

In [38]:
val b3 = isBST_Efficient(tree1)

b3: Boolean = false

In [39]:
val b4 = isBST_Efficient(tree2)

b4: Boolean = true

## Arithmetic Expressions



Arithmetic expressions are generated by the grammar we saw before.

$$\begin{array}{rcll}
\textbf{Expr} & \rightarrow & Const(\textbf{Double}) \\
& |  & Ident(\textbf{Identifier}) \\
& | & Plus( \textbf{Expr}^+) & \text{Note:}\ A^+ \ \text{is one or more reps of}\ A \\
& | & Minus( \textbf{Expr}, \textbf{Expr}^+) \\
& | & Mult(\textbf{Expr}^+) \\
& | & Div(\textbf{Expr}, \textbf{Expr}) \\
& | & Log(\textbf{Expr}) \\
& | & Exp(\textbf{Expr}) \\
& | & Sine(\textbf{Expr}) \\
& | & Cosine(\textbf{Expr}) \\\\
\textbf{Double} & \rightarrow & \text{all double precision numbers in Scala}\\
\textbf{Identifier} & \rightarrow & [a-zA-Z][a-z\ A-Z\ 0-9\ \_]* & \text{Note: All strings that begin with letters}\\
&&& \text{a-z or A-Z and subsequently can contain a-z, A-Z, 0-9 or _ chars}
\end{array}$$

In [1]:
sealed trait Expr
case class Const(f: Double) extends Expr 
// 1. We cheated a bit and allowed all floating point numbers
// Also, this deviates from the grammar
case class Ident(s: String) extends Expr
// 2. We allow any string to be an identifier for now instead of the regular expression shown in the grammar.
case class Plus( eList: List[Expr] ) extends Expr
case class Minus(e1: Expr, eList: List[Expr]) extends Expr
case class Mult(eList: List[Expr]) extends Expr
case class Div(e1: Expr, e2: Expr) extends Expr
case class Log(e: Expr) extends Expr
case class Exp(e: Expr) extends Expr
case class Sine(e: Expr) extends Expr
case class Cosine(e: Expr) extends Expr

defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Div
defined class Log
defined class Exp
defined class Sine
defined class Cosine

The simplest function we wish to write will collect all the variables that are used in a given expression inside a Set data structure. Read about scala sets here: https://docs.scala-lang.org/overviews/collections/sets.html



In [41]:
def collectAllVariables(e: Expr): Set[String] = {
    def helperFunction(eList: List[Expr], startValue: Set[String]): Set[String] = {
        // Iterate through the list eList and collect set of variables for each expression in eList.
        // Next: take the union of all the sets with the given set startValue.
            eList.foldLeft (startValue) {
                case (setSoFar: Set[String], newExpr:Expr) => { val v = collectAllVariables(newExpr)
                                              setSoFar union v
                                            }
        
            }
        // If you insist on a for loop, you have to use a var
        // var setSoFar = startValue
        // for (newExpr <- eList) {
        //   setSoFar = setSoFar.union(collectAllVariables(newExpr))
        //    }
        // return setSoFar
    }
        
    e match {
        case Const(f) => Set() // Return the empty set
        case Ident(s) => Set(s) // Return a singleton set with s in it.
        case Plus(eList) => helperFunction(eList, Set())  //Union of all variables in eList
        case Mult(eList) => helperFunction(eList, Set()) // Union of all variables in eList
    
        case Minus(e1, eList) => {
            val e1Set = collectAllVariables(e1) // First collect variables in e1
            helperFunction(eList, e1Set) // take union of all variables in eList with e1Set from prev. step
        }
    
    
        case Div(e1, e2) => {
            (collectAllVariables(e1)) union (collectAllVariables(e2))
        }
        case Log(e1) => collectAllVariables(e1)
        case Sine(e1) => collectAllVariables(e1)
        case Cosine(e1) => collectAllVariables(e1)
        case Exp(e1) => collectAllVariables(e1)
        case _ => { assert(false); return Set()  } // This is the catch all. Should not happen but good coding practice
    }
}  

defined function collectAllVariables

In [42]:
// cos(x) + sin(y) + e^{x - y - Zzz}
val x = Ident("x")
val y = Ident("y")
val z = Ident("Zzz")
val expr1 = Plus(List(Cosine(x), Sine(y), Exp(Minus(x, List(y,z)))))

x: Ident = Ident("x")
y: Ident = Ident("y")
z: Ident = Ident("Zzz")
expr1: Plus = Plus(
  List(
    Cosine(Ident("x")),
    Sine(Ident("y")),
    Exp(Minus(Ident("x"), List(Ident("y"), Ident("Zzz"))))
  )
)

In [43]:
val s1 = collectAllVariables(expr1)

s1: Set[String] = Set("x", "y", "Zzz")

In [44]:
val expr2 = Plus(List(Exp(Const(1.5)), Sine(Const(2.0)), Cosine(Mult(List(Const(1.2), Const(2.4))))))

expr2: Plus = Plus(
  List(
    Exp(Const(1.5)),
    Sine(Const(2.0)),
    Cosine(Mult(List(Const(1.2), Const(2.4))))
  )
)

In [45]:
val s2 = collectAllVariables(expr2)

s2: Set[String] = Set()

Another important function is pretty print. We would like to pretty print our expression to Latex so that we can cut and paste it in our documents.

Latex allows you to type formulae and have them displayed. Eg., let us take the formula

$$ \frac{M c^2}{ 1 + \frac{1}{1 + c^2}}$$

You would write it as

\$ \\frac{M c^2}{ 1 + \\frac{1}{1 + c^2} \$

In [46]:
def toLatex (e: Expr): String = {
    e match {
        case Const(f) => "%2.2f".format(f) // easy way to convert f to a string.
        case Ident(str) => str
        case Plus(eList) => { val strList = eList map { e =>  toLatex(e) }
                         "\\left("+ strList.mkString (" + ") + "\\right)" // Join each expression into a string separated by " + "
                       }
        case Minus(e, eList) => {
            val s1 = toLatex(e)
            val strList = eList map {e => toLatex(e)} 
             "\\left("+ s1 + " - " + (strList mkString " - ")+ "\\right)" 
            
        }
        case Mult(eList) => {  "\\left(" + (eList map {toLatex(_)} mkString ("\\ ")) + "\\right)"  }
        
        case Div(e1, e2) => { "\\frac{" + (toLatex(e1)) + "}{" + (toLatex(e2)) + "}" }
        case Log(e) => { "\\log(" + (toLatex(e)) + ")" }
        case Exp(e) => { "e^{" + (toLatex(e)) + "}"}
        case Sine(e) => { "\\sin\\left("+ toLatex(e) + "\\right)" }
        case Cosine(e) => {"\\cos\\left("+ toLatex(e) + "\\right)" }
    }
    
}

defined function toLatex

In [47]:
val str1= "$"+toLatex(expr1)+"$"
print(str1)
latex(str1)

$\left(\cos\left(x\right) + \sin\left(y\right) + e^{\left(x - y - Zzz\right)}\right)$

str1: String = "$\\left(\\cos\\left(x\\right) + \\sin\\left(y\\right) + e^{\\left(x - y - Zzz\\right)}\\right)$"

In [48]:
val w = Ident("w")
val expr2 = Div(Minus( Plus(List(w, Mult(List(x, y)), 
                                Plus(List(Const(1.2), w))) ), 
                                     List(Const(1.5), w, x) ), Const(2.3))
latex("$"+ toLatex(expr2) + "$")


w: Ident = Ident("w")
expr2: Div = Div(
  Minus(
    Plus(
      List(
        Ident("w"),
        Mult(List(Ident("x"), Ident("y"))),
        Plus(List(Const(1.2), Ident("w")))
      )
    ),
    List(Const(1.5), Ident("w"), Ident("x"))
  ),
  Const(2.3)
)

### My First Eval Function

Now we will write a very important operation called `eval`. This operation is the primary operation that an interpreter needs to carry out. 

But we will start small by writing an eval for our simple expression grammar. Before we do that, let us ask ourselves two questions: (a) what does it mean for the expressions to be evaluated? and (b) what is the missing information that we need before we can evaluate our arithmetic expressions defined above?
The answers should be easy in this case:

1. An arithmetic expression stands for a number which is the answer when we calculate it and 
2. Of course, the missing information is that we do not know what the valuations for the variables are to start with.

Thus, we are ready to define the eval function.

__Environment__ 

An environment $\eta$ maps every identifier string to a numeric value. 


Example: 

$$ \{ x \mapsto 2, y \mapsto 1.5, Zzz \mapsto 2.8, w \mapsto 15.2 \}$$

Scala has a Map data structure that is ideal for environments

In [49]:
val myEnvironment: Map[String, Double] = Map("x" -> 2.0, "y" -> 1.5, "Zzz" -> 2.8, "w" -> 15.2, "l" -> 129.3)

myEnvironment: Map[String, Double] = Map(
  "x" -> 2.0,
  "y" -> 1.5,
  "Zzz" -> 2.8,
  "l" -> 129.3,
  "w" -> 15.2
)

In [50]:
def evalExpr (e: Expr, env: Map[String, Double]): Double = e match {
    case Const (f) => f
    case Ident (str) => { if (env.contains(str)){
                                env(str)
                            } else {
        throw new IllegalArgumentException(s"Environment does not contain mapping for $str")
    }
                        }
    case Plus(eList) => {
        (eList map {evalExpr(_, env)}).sum
    }
    
    case Minus(e, eList) => {
        (evalExpr(e, env)) - ((eList map {evalExpr(_, env)}).sum )
    }
    
    case Mult(eList) => {
        (eList map {evalExpr(_, env)}).product
    }
    
    case Div(e1, e2) => {
        (evalExpr(e1, env)) / (evalExpr(e2, env))
    }
    
    case Log(e) => math.log(evalExpr(e, env))
    
    
    case Exp(e) => math.exp( evalExpr(e, env))
    
    case Sine(e) => math.sin( evalExpr(e, env))
    
    case Cosine(e) => math.cos(evalExpr(e, env))
}

defined function evalExpr

In [51]:
val f1 = evalExpr(expr1, myEnvironment)

f1: Double = 0.6816069937797158

In [52]:
val f2 = evalExpr(expr2, myEnvironment)

f2: Double = 6.913043478260868

In [53]:
val expr3 = Minus(Const(1.0), List(
    Mult(List(Sine(x), Sine(x))),
    Mult(List(Cosine(x), Cosine(x)))
))

latex("$"+toLatex(expr3)+"$")

expr3: Minus = Minus(
  Const(1.0),
  List(
    Mult(List(Sine(Ident("x")), Sine(Ident("x")))),
    Mult(List(Cosine(Ident("x")), Cosine(Ident("x"))))
  )
)

In [54]:
val f3 = evalExpr(expr3, myEnvironment) // Should be 0.0

f3: Double = 0.0

In [55]:
val f4 = Log(Plus(List(Const(1.0), Mult(List(Const(2.0), Ident("y"))))))
val env = Map("y" -> 1.2)

f4: Log = Log(Plus(List(Const(1.0), Mult(List(Const(2.0), Ident("y"))))))
env: Map[String, Double] = Map("y" -> 1.2)

In [56]:
val f5 = evalExpr(f4,env )

f5: Double = 1.2237754316221157

We learned how to implement some operations on our inductive data structures.
Now we are ready in the next step to learn about how to write an interpreter and start adding features to the interpreter. But you may have already sensed something important: we have developed the bad habit of writing
lots of code to carry out various operations and using the code as the definition. This works for simple operations which are easy for us to understand. For a more complex beast like a program, we will need a disciplined approach to understanding and writing down the workings of the program outside of code and in a way that can be analyzed by people. We will learn these as operational semantics in the coming lectures.